In [9]:
def lire_fichier(chemin):
  f = open(chemin)
  chaine = f.read()
  f.close()
  return chaine

def decouper_en_mots(chaine):
  liste_mots = chaine.split()
  return liste_mots

def evaluer(VP, FN, FP):
  if VP!=0:
    rappel = VP/(VP+FN)
    precision=VP/(VP+FP)
    f_mesure = (2*rappel*precision)/(precision+rappel)
  else:
    rappel, precision, f_mesure = 0, 0, 0
  return rappel, precision, f_mesure

import os, glob, json
try:
  os.makedirs("Modeles")
except:
  pass


In [5]:
for N in range(1, 21):
  print("Apprentissage pour NB mots = %i"%N)
  dic_langues = {}
  for chemin in glob.glob("corpus_multi/*/appr/*"):
    dossiers = chemin.split("/")#à adapter
    langue = dossiers[1]
    dic_langues.setdefault(langue, {})#refactor
    chaine = lire_fichier(chemin)
    mots = chaine.split()
    for m in mots:
      dic_langues[langue].setdefault(m, 0) #refactor
      dic_langues[langue][m] += 1
  dic_models = {}
  for langue, dic_effectifs in dic_langues.items():
    paires = [[effectif, mot] for mot, effectif in  dic_effectifs.items()]
    liste_tri = sorted(paires)[-N:]#les N mots fréquents
    dic_models[langue] = [mot for effectif, mot in liste_tri]
  w = open("Modeles/models_NBmots=%i.json"%N, "w")
  w.write(json.dumps(dic_models, indent = 2))
  w.close()

Apprentissage pour NB mots = 1
Apprentissage pour NB mots = 2
Apprentissage pour NB mots = 3
Apprentissage pour NB mots = 4
Apprentissage pour NB mots = 5
Apprentissage pour NB mots = 6
Apprentissage pour NB mots = 7
Apprentissage pour NB mots = 8
Apprentissage pour NB mots = 9
Apprentissage pour NB mots = 10
Apprentissage pour NB mots = 11
Apprentissage pour NB mots = 12
Apprentissage pour NB mots = 13
Apprentissage pour NB mots = 14
Apprentissage pour NB mots = 15
Apprentissage pour NB mots = 16
Apprentissage pour NB mots = 17
Apprentissage pour NB mots = 18
Apprentissage pour NB mots = 19
Apprentissage pour NB mots = 20


In [12]:
dic_courbes = {}#pour faire des courbes ultérieurement
vp, fp, fn = 0, 0, 0
for N in range(1, 21):
  print("Test pour NB mots = %i"%N)
  f = open("Modeles/models_NBmots=%i.json"%N, "r")
  dic_models = json.load(f)
  f.close()
  liste_langues = dic_models.keys() 
  dic_courbes[N] = {lg : [[], [], []] for lg in liste_langues} 
  dic_resultats ={lg: {"VP":0, "FP":0, "FN":0} for lg in liste_langues}
  for chemin in glob.glob("corpus_multi/*/test/*"):
    dossiers = chemin.split("/")
    langue = dossiers[1]
    chaine = lire_fichier(chemin)
    mots = chaine.split()
    dic_freq_texte = {}
    for m in mots:
      dic_freq_texte.setdefault(m, 0) #refactor
      dic_freq_texte[m] += 1
    paires = [[effectif, mot] for mot, effectif in  dic_freq_texte.items()]
    liste_tri = sorted(paires)[-N:]#les N mots plus fréquents
    plus_frequents = set([mot for effectif, mot in liste_tri])
    liste_predictions = []
    for langue_ref, model in dic_models.items():
      mots_communs = set(model).intersection(plus_frequents)
      liste_predictions.append([len(mots_communs), langue_ref])#refactor
    lg_pred = sorted(liste_predictions)[-1][1]#refactor
    if lg_pred == langue:
      dic_resultats[langue]["VP"]+=1
      vp+=1
    else:
      dic_resultats[langue]["FN"]+=1
      dic_resultats[lg_pred]["FP"]+=1
      fp+=1
      fn+=1
  for langue, infos in dic_resultats.items():
    VP = infos["VP"]
    FP = infos["FP"]
    FN = infos["FN"]
    rappel, precision, f_mesure = evaluer(VP, FN, FP)

    #print("%s : rappel =%f, précision=%f et f-mesure=%f"%(langue,rappel, precision, f_mesure))#par langue
    dic_courbes[N][langue][0].append(rappel)
    dic_courbes[N][langue][1].append(precision)
    dic_courbes[N][langue][2].append(f_mesure)
  rappel, precision, f_mesure= evaluer(vp, fn, fp)#pour toutes les langues cette fois
  print("  ALL : rappel =%f, précision=%f et f-mesure=%f"%(rappel, precision, f_mesure))
w = open("resultats_mots.json", "w")
w.write(json.dumps(dic_courbes, indent = 2))
w.close()

Test pour NB mots = 1
  ALL : rappel =0.467532, précision=0.467532 et f-mesure=0.467532
Test pour NB mots = 2
  ALL : rappel =0.574269, précision=0.574269 et f-mesure=0.574269
Test pour NB mots = 3
  ALL : rappel =0.646916, précision=0.646916 et f-mesure=0.646916
Test pour NB mots = 4
  ALL : rappel =0.701907, précision=0.701907 et f-mesure=0.701907
Test pour NB mots = 5
  ALL : rappel =0.741883, précision=0.741883 et f-mesure=0.741883
Test pour NB mots = 6
  ALL : rappel =0.775162, précision=0.775162 et f-mesure=0.775162
Test pour NB mots = 7
  ALL : rappel =0.801252, précision=0.801252 et f-mesure=0.801252
Test pour NB mots = 8
  ALL : rappel =0.821631, précision=0.821631 et f-mesure=0.821631
Test pour NB mots = 9
  ALL : rappel =0.837753, précision=0.837753 et f-mesure=0.837753
Test pour NB mots = 10
  ALL : rappel =0.850812, précision=0.850812 et f-mesure=0.850812
Test pour NB mots = 11
  ALL : rappel =0.861423, précision=0.861423 et f-mesure=0.861423
Test pour NB mots = 12
  ALL :